In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import sqlite3
import pandas as pd
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, pipeline

cnx = sqlite3.connect('/content/drive/My Drive/CAFE달달.db') # db 파일 경로 입력
dataCSV = pd.read_sql_query("SELECT * FROM crowling_all", cnx)

cnx.commit()
cnx.close()

def use_split_join(data):
    string = ' '.join(data.split())
    return string


dataCSV['comment'] = dataCSV['comment'].str.join('').str.replace(r"\n", "") # 줄바꿈 제거

# 전처리
dataCSV['comment'] = dataCSV['comment'].str.replace(pat=r'[^\w]', repl=r' ', regex=True) # 특수문자를 공백으로
dataCSV['comment'] = dataCSV['comment'].apply(use_split_join) # 여러개 공백을 하나로 바꿈

example = dataCSV['comment'].tolist()



## sentiment analysis
koelectra_finetuned_model_dir = os.path.join('/content/drive/My Drive/',"koelectra-review-finetune.bin")
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")
model.load_state_dict(torch.load(koelectra_finetuned_model_dir, map_location=torch.device('cpu')))
sentiment_classifier = pipeline('sentiment-analysis', tokenizer=tokenizer, model=model)


y_pred = []
total_len = len(example)
for cnt, review in enumerate(example):
    print(review)
    pred = sentiment_classifier(review)
    print(f"{cnt} / {total_len} : {pred[0]}")
    if pred[0]['label'] == 'LABEL_1':
        y_pred.append(1)
    else:
        y_pred.append(0)



Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

티라미수 오랜만에 먹었는데 맛있네요 사장님 다른 과자 쪼꼬렛은 괜찮거든요 ㅎㅎ
0 / 16 : {'label': 'LABEL_0', 'score': 0.9980216026306152}
잘 마셨습니다
1 / 16 : {'label': 'LABEL_0', 'score': 0.9984897375106812}
배X에 한라봉 신메뉴 올라와서 일단 시켰네요ㅎㅎ라봉라봉 한라봉 너무 맛있었어요 여기선 오렌지로 주전부리 손이가요 손이가 잘마셨습니다
2 / 16 : {'label': 'LABEL_0', 'score': 0.9976053237915039}
늘 양도많고 맛있어요
3 / 16 : {'label': 'LABEL_0', 'score': 0.9986112117767334}
커피진하고 좋아요 빵도 맛있고 서비스로 주신 쿠키도 맛있었습니다 감사합니다 또 시킬게요
4 / 16 : {'label': 'LABEL_1', 'score': 0.9989235997200012}
오우우 호박식혜 참 맛있네요 다른 메뉴들은 당연히 맛있고요 서비스로 주신 프레첼도 맛나서 아껴먹었어여 ㅎㅎ
5 / 16 : {'label': 'LABEL_1', 'score': 0.9989266991615295}
자주 시켜먹던곳인데 한동안 배달앱에 안보여서 아쉬웠는데 다시 보여서 바로 시켰어요 딸기라떼가 대존맛
6 / 16 : {'label': 'LABEL_0', 'score': 0.9982170462608337}
서비스 무슨일이에요 소소한 행복 배달 빨랐구요 민트초코는 생각보다 단편이지만 부드럽고 맛있어요 치즈케익도 가격대비 짱 짱
7 / 16 : {'label': 'LABEL_1', 'score': 0.9989336133003235}
맛있어오 주변인가봐요 배송개빠룸
8 / 16 : {'label': 'LABEL_0', 'score': 0.9984706044197083}
오랜지주스 개크네염 ㅋㄷㅋㄷ
9 / 16 : {'label': 'LABEL_0', 'score': 0.9984727501869202}
과자도 서